## Data Loading

In [1]:
import pandas as pd
import numpy as np
import sklearn
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

books = pd.read_csv("data/books.csv")
books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [2]:
# Only keep necessary columns for Books DataFrame
cols = ['book_id', 'isbn', 'authors', 'title', 'language_code', 'average_rating', 'ratings_count']
books = books[cols]

# Only keep books that are in English
books = books.loc[books['language_code'].isin(['eng', 'en-US', 'en-CA', 'en-GB'])]
books.head()

,book_id,isbn,authors,title,language_code,average_rating,ratings_count
0,1,439023483,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653
1,2,439554934,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479
2,3,316015849,Stephenie Meyer,"Twilight (Twilight, #1)",en-US,3.57,3866839
3,4,61120081,Harper Lee,To Kill a Mockingbird,eng,4.25,3198671
4,5,743273567,F. Scott Fitzgerald,The Great Gatsby,eng,3.89,2683664


In [3]:
ratings = pd.read_csv("data/ratings.csv")
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


## Recommendation System using kNN

#### Idea: Represent ratings as a matrix M
* Each row is a book, each column is a user 
* M[i][j] = k -- means user j rated book i with k stars

#### First, combine books and ratings based on book_id

In [4]:
combine_book_rating = pd.merge(ratings, books, on='book_id')
columns = ['user_id', 'book_id', 'rating', 'title']
combine_book_rating = combine_book_rating[columns]
print(combine_book_rating.shape)
combine_book_rating.head()

(5577975, 4)


,user_id,book_id,rating,title
0,1,258,5,The Shadow of the Wind (The Cemetery of Forgot...
1,11,258,3,The Shadow of the Wind (The Cemetery of Forgot...
2,143,258,4,The Shadow of the Wind (The Cemetery of Forgot...
3,242,258,5,The Shadow of the Wind (The Cemetery of Forgot...
4,325,258,4,The Shadow of the Wind (The Cemetery of Forgot...


#### Get total number of ratings per book

In [5]:
ratings_count = combine_book_rating.groupby(by=['title'])['rating'].count().reset_index()
ratings_count = ratings_count.rename(columns = {'rating':'ratings_count'})
ratings_count.head()

,title,ratings_count
0,"Angels (Walsh Family, #3)",263
1,#GIRLBOSS,195
2,'Salem's Lot,4433
3,"'Tis (Frank McCourt, #2)",703
4,"10% Happier: How I Tamed the Voice in My Head,...",297


In [6]:
rating_with_count = combine_book_rating.merge(ratings_count, left_on='title', right_on='title', how='left')
rating_with_count.head()

,user_id,book_id,rating,title,ratings_count
0,1,258,5,The Shadow of the Wind (The Cemetery of Forgot...,3531
1,11,258,3,The Shadow of the Wind (The Cemetery of Forgot...,3531
2,143,258,4,The Shadow of the Wind (The Cemetery of Forgot...,3531
3,242,258,5,The Shadow of the Wind (The Cemetery of Forgot...,3531
4,325,258,4,The Shadow of the Wind (The Cemetery of Forgot...,3531


#### Remove rows that have the same user_id and book_id

In [7]:
if not rating_with_count[rating_with_count.duplicated(['user_id', 'title'])].empty:
    initial_rows = rating_with_count.shape[0]
    print('Initial dataframe shape {0}'.format(rating_with_count.shape))
    rating_with_count = rating_with_count.drop_duplicates(['user_id', 'title'])
    current_rows = rating_with_count.shape[0]
    print('New dataframe shape {0}'.format(rating_with_count.shape))
    print('Removed {0} rows'.format(initial_rows - current_rows))

Initial dataframe shape (5577975, 5)
New dataframe shape (5577229, 5)
Removed 746 rows


#### Pivoting: book title vs. user ID

In [8]:
# For memory limitations, select the first 1 million ratings only
rating_with_count = rating_with_count[:1000000]
rating_pivot = rating_with_count.pivot(index='title', columns='user_id', values='rating').fillna(0)
rating_pivot.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,53414,53415,53417,53418,53419,53420,53421,53422,53423,53424
title,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1984,4.0,0.0,2.0,4.0,0.0,0.0,5.0,4.0,5.0,0.0,...,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
"A Child Called ""It"" (Dave Pelzer #1)",0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
A Confederacy of Dunces,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Dangerous Fortune,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Train kNN model
#### Distance metric: Cosine similarity

In [9]:
rating_matrix = csr_matrix(rating_pivot.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(rating_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

### Get book recommendations!

In [10]:
def get_recommendations(book_title, topn=6):
    book_index = list(rating_pivot.index).index(book_title)
    distances, indices = model_knn.kneighbors(rating_pivot.iloc[book_index,:].values.reshape(1,-1), n_neighbors=topn)
    print('Recommendations for {}:'.format(rating_pivot.index[book_index]))
    for i in range(1, len(distances.flatten())):
        print('{}. {}, distance = {}'.format(i, rating_pivot.index[indices.flatten()[i]], "%.3f"%distances.flatten()[i]))
    print()
    
get_recommendations("Harry Potter and the Sorcerer's Stone (Harry Potter, #1)")
get_recommendations("Moby-Dick or, The Whale")
get_recommendations("Angels & Demons  (Robert Langdon, #1)")
get_recommendations("Charlie and the Chocolate Factory (Charlie Bucket, #1)")

Recommendations for Harry Potter and the Sorcerer's Stone (Harry Potter, #1):
1. Harry Potter and the Prisoner of Azkaban (Harry Potter, #3), distance = 0.320
2. Harry Potter and the Chamber of Secrets (Harry Potter, #2), distance = 0.327
3. Harry Potter and the Goblet of Fire (Harry Potter, #4), distance = 0.331
4. Harry Potter and the Order of the Phoenix (Harry Potter, #5), distance = 0.343
5. Harry Potter and the Half-Blood Prince (Harry Potter, #6), distance = 0.348

Recommendations for Moby-Dick or, The Whale:
1. The Adventures of Huckleberry Finn, distance = 0.703
2. Frankenstein, distance = 0.710
3. The Old Man and the Sea, distance = 0.729
4. Animal Farm, distance = 0.734
5. 1984, distance = 0.739

Recommendations for Angels & Demons  (Robert Langdon, #1):
1. The Da Vinci Code (Robert Langdon, #2), distance = 0.452
2. Deception Point, distance = 0.534
3. Harry Potter and the Sorcerer's Stone (Harry Potter, #1), distance = 0.559
4. The Kite Runner, distance = 0.626
5. To Kill a

##### Our model give relevant recommendations for books of different genres!